In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin


  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-d9h650mq
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-d9h650mq
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=0551b696fe48171ca5641908b280de396351b2c4ed9aaf684cef3f8a7f7433b8
  Stored in directory: /tmp/pip-ephem-wheel-cache-f8bd3k01/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <cstdio>
#include <math.h>
#include <stdlib.h>
#include <stdio.h>
#include <string.h>

__global__ void findWordCount(char* str, char* word, int* count, int strl, int wordl)
{
	//here  every thread  will skip the previous words and check for only its words
  // for that thread only 
	int id = threadIdx.x;
	int c = 0;
	int i = 0;
	for (i = 0; i < strl; i++)
	{
		if (str[i] == ' ')
			c++;
		if (c == id)
		{
			break;
		}
	}
	char* w = (char*)malloc(sizeof(char) * 30);
	int j = 0;
	if (i != 0)
		i = i + 1;
	for (;i < strl;i++)
	{
		if (str[i] == ' ')
			break;
		w[j++] = str[i];
	}
	w[j] = '\0';
	printf("Current Thread --> %d\n , and the current word checking is %s\n",id,w);
    
	int flag = 0;
	for (i = 0;i < j;i++)
		if (word[i] != w[i])
		{
		    flag = 1;
           break;
		}
	if (flag == 0){
		atomicAdd(count, 1);
    }
}

int main()
{
	char* string1 = (char*)malloc(sizeof(char) * 300);
	char* pattern = (char*)malloc(sizeof(char) * 30);
	string1 = "LAMBDA CALCULAS IS DEAD LONG LIVE LAMBDA";
	pattern = "LAMBDA";
	int* wordCount = (int*)malloc(sizeof(int));
	*wordCount = 0;
	printf("Given String: %s\n ",string1);
    printf("Given Word: %s\n", pattern);
	
    // Count the number of words in the string
	
    int count = 1; //For the last word
	for (int i = 0;i < strlen(string1);i++)
	{
		if (string1[i] == ' ')
			count++;
	}
	char* d_str, * d_word;
	int* d_wordCount;
	
    cudaMalloc((void**)&d_str, sizeof(char) * 300);
	cudaMalloc((void**)&d_word, sizeof(char) * 30);
	cudaMalloc((void**)&d_wordCount, sizeof(int));
	
    cudaMemcpy(d_str, string1, sizeof(char) * 300, cudaMemcpyHostToDevice);
	cudaMemcpy(d_word, pattern, sizeof(char) * 30, cudaMemcpyHostToDevice);
	cudaMemcpy(d_wordCount, wordCount, sizeof(int), cudaMemcpyHostToDevice);
	
    dim3 gridDim(1, 1, 1);
	dim3 blockDim(count, 1, 1);
	
    findWordCount << < 1, count >> > (d_str, d_word, d_wordCount, strlen(string1), strlen(pattern));
	cudaMemcpy(wordCount, d_wordCount, sizeof(int), cudaMemcpyDeviceToHost);
	printf("Total count of %s in %s is : %d\n", pattern, string1,*wordCount);
	
    cudaFree(d_word);
	cudaFree(d_wordCount);
	cudaFree(d_str);
	
    
    return 0;
}

Given String: LAMBDA CALCULAS IS DEAD LONG LIVE LAMBDA
 Given Word: LAMBDA
Current Thread --> 0
 , and the current word checking is LAMBDA
Current Thread --> 1
 , and the current word checking is CALCULAS
Current Thread --> 2
 , and the current word checking is IS
Current Thread --> 3
 , and the current word checking is DEAD
Current Thread --> 4
 , and the current word checking is LONG
Current Thread --> 5
 , and the current word checking is LIVE
Current Thread --> 6
 , and the current word checking is LAMBDA
Total count of LAMBDA in LAMBDA CALCULAS IS DEAD LONG LIVE LAMBDA is : 2

